In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io as io
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

import os.path as osp
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame

/opt/conda/envs/mmdet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## 사용한 val_fold json file 경로
GT_JSON = '/opt/ml/baseline/level2-object-detection-level2-cv-19/DATAResearch/inference/serverbase_submission.json'

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

## inference batch size 지정
BATCH_SIZE = 32

## 원하는 config, checkpoint 경로
config_path = 'configs 폴더에 들어가 있는 .py 경로'
checkpoint_path = '.pth 파일 경로'

In [4]:
gt = []

coco = COCO(GT_JSON)
   
'''
image id 별로 GT 값 추가 (여기서는 test.json을 사용하였기 때문에 GT가 없습니다.)
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                    float(annotation['socre']),
                    float(annotation['bbox'][0]),
                    float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                    float(annotation['bbox'][1]),
                    (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=1.04s)
creating index...
index created!


In [6]:
image_info = coco.loadImgs(0)[0]
annotation_id = coco.getAnnIds(imgIds=image_info['id'])
annotation_info_list = coco.loadAnns(annotation_id)

In [7]:
annotation_info_list

[{'image_id': 0,
  'category_id': 7,
  'score': 0.998166561126709,
  'area': 98200.27337541606,
  'bbox': [219.02938842773438,
   55.14566421508789,
   236.60061645507812,
   415.0465660095215],
  'iscrowd': 0,
  'id': 0},
 {'image_id': 0,
  'category_id': 7,
  'score': 0.9964185953140259,
  'area': 172733.77079609782,
  'bbox': [601.6265258789062,
   522.7828979492188,
   347.1573486328125,
   497.56622314453125],
  'iscrowd': 0,
  'id': 1},
 {'image_id': 0,
  'category_id': 7,
  'score': 0.9941514134407043,
  'area': 64156.72965930635,
  'bbox': [118.00577545166016,
   428.7281494140625,
   215.83269500732422,
   297.25213623046875],
  'iscrowd': 0,
  'id': 2},
 {'image_id': 0,
  'category_id': 7,
  'score': 0.9896167516708374,
  'area': 84082.2263385728,
  'bbox': [583.9705200195312,
   325.1666259765625,
   293.3470458984375,
   286.63055419921875],
  'iscrowd': 0,
  'id': 3},
 {'image_id': 0,
  'category_id': 7,
  'score': 0.9893291592597961,
  'area': 54774.65197169781,
  'bbox':

In [5]:
gt

[['test/0000.jpg',
  7,
  219.02938842773438,
  455.6300048828125,
  55.14566421508789,
  470.1922302246094],
 ['test/0000.jpg',
  7,
  601.6265258789062,
  948.7838745117188,
  522.7828979492188,
  1020.34912109375],
 ['test/0000.jpg',
  7,
  118.00577545166016,
  333.8384704589844,
  428.7281494140625,
  725.9802856445312],
 ['test/0000.jpg',
  7,
  583.9705200195312,
  877.3175659179688,
  325.1666259765625,
  611.7971801757812],
 ['test/0000.jpg',
  7,
  444.8504943847656,
  647.8341064453125,
  607.2685546875,
  877.1162109375],
 ['test/0000.jpg',
  1,
  558.3930053710938,
  785.3493041992188,
  109.7670669555664,
  368.85833740234375],
 ['test/0000.jpg',
  7,
  387.65625,
  600.2269287109375,
  203.0829315185547,
  546.9764404296875],
 ['test/0000.jpg',
  7,
  256.0802917480469,
  431.71038818359375,
  429.7945556640625,
  579.935546875],
 ['test/0000.jpg',
  7,
  557.973876953125,
  706.3729248046875,
  347.25933837890625,
  605.35546875],
 ['test/0000.jpg',
  1,
  498.953399658